**Done by Subhajit**

In [ ]:
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten

In [ ]:
import tensorflow as tf

In [ ]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 300
num_filter = 35
compression = 1

In [ ]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

170498071/170498071 [==============================] - 3s 0us/step


In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
traindataGen=ImageDataGenerator(shear_range=0.2,zoom_range=0.2, rotation_range=20, width_shift_range=0.2,
height_shift_range=0.2,horizontal_flip=True,fill_mode='nearest')
testdataGen=ImageDataGenerator()

In [ ]:
traindataGen.fit(X_train)

In [ ]:
print('X_train shape =', X_train.shape)
print('X_test shape =', X_test.shape)
print('y_train shape =', y_train.shape)
print('y_test shape =', y_test.shape)

X_train shape = (50000, 32, 32, 3)
X_test shape = (10000, 32, 32, 3)
y_train shape = (50000, 10)
y_test shape = (10000, 10)


In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint
import datetime

In [ ]:
import os
os.mkdir('model_save')

In [ ]:
import tensorflow.keras.backend as k
def changeLearningRate(epoch):
  ''' This function changes learing rate based on validation accuracy and epoch number'''
  learning_rate = k.eval(model.optimizer.lr)
  if epoch == epochs/2:
     #The learning rate is divided by 10 at 50% of the total number of training epochs
    learning_rate = learning_rate * 0.1
   #The learning rate is divided by 10 at 75% of the total number of training epochs
  elif epoch == (epochs*3)/4:
      learning_rate = learning_rate * 0.1
  return learning_rate

In [ ]:
# Dense Block
def denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])

        temp = concat

    return temp

## transition Blosck
def transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [ ]:
l = 6
dropout_rate = 0
input = layers.Input(shape=(img_height, img_width, channel))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate=0)
First_Transition = transition(First_Block, num_filter, dropout_rate=0)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate=0)
Second_Transition = transition(Second_Block, num_filter, dropout_rate=0)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate=0)
Third_Transition = transition(Third_Block, num_filter, dropout_rate=0)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate=0)
output = output_layer(Last_Block)

In [ ]:
model = Model(inputs=input, outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 35)   945         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 35)  140         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 35)   0           ['batch_normalization[0][0]']

In [ ]:
model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True),
                loss=tf.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

In [ ]:
#lrschedule = LearningRateScheduler(changeLearningRate, verbose=1)
filepath = 'model_save/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')
callback_list = [checkpoint]

In [ ]:
model.fit(traindataGen.flow(X_train, y_train, batch_size=128), steps_per_epoch=len(X_train) / 128, epochs=epochs, validation_data=testdataGen.flow(X_test, y_test), callbacks=callback_list)

Epoch 1/300


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5561: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


391/390 [==============================] - ETA: 0s - loss: 1.6051 - accuracy: 0.4134
Epoch 1: val_accuracy improved from -inf to 0.34000, saving model to model_save/weights-01-0.3400.hdf5
390/390 [==============================] - 84s 164ms/step - loss: 1.6051 - accuracy: 0.4134 - val_loss: 1.8068 - val_accuracy: 0.3400
Epoch 2/300
391/390 [==============================] - ETA: 0s - loss: 1.2609 - accuracy: 0.5457
Epoch 2: val_accuracy improved from 0.34000 to 0.54020, saving model to model_save/weights-02-0.5402.hdf5
390/390 [==============================] - 60s 154ms/step - loss: 1.2609 - accuracy: 0.5457 - val_loss: 1.4040 - val_accuracy: 0.5402
Epoch 3/300
391/390 [==============================] - ETA: 0s - loss: 1.0684 - accuracy: 0.6198
Epoch 3: val_accuracy improved from 0.54020 to 0.61480, saving model to model_save/weights-03-0.6148.hdf5
390/390 [==============================] - 62s 158ms/step - loss: 1.0684 - accuracy: 0.6198 - val_loss: 1.0812 - val_accuracy: 0.6148
Epoc